# Parte 01: Portada y Entorno  

## Magíster en Ciencia de Datos – UC  
**Asignatura:** Aprendizaje Estadístico y Computacional  
**Tarea:** Aprendizaje Supervisado y No Supervisado  


## Objetivo de la Tarea  
Aplicar técnicas de aprendizaje supervisado y no supervisado sobre un dataset real de clientes de tarjetas de crédito (UCI Credit Card Default), con el fin de:  

- Predecir el riesgo de impago (default).  
- Identificar segmentos de clientes según su comportamiento de pago.  
- Integrar los hallazgos en un informe académico y un notebook documentado.  


## Contexto Académico  
- Tipo de actividad: Grupal  
- Evaluación: Sumativa  
- Ponderación: 20% de la asignatura  
- Puntaje total: 60 puntos  
- Exigencia mínima: 50% (nota mínima 4.0)  


## Evaluación  
- Revisión mediante notebook ejecutable y un informe en formato APA (2.500–7.500 palabras).  
- Se evaluará:  
  - Rigor metodológico (validación, resultados, reproducibilidad).  
  - Interpretabilidad y aplicabilidad de los hallazgos en un contexto crediticio.  
  - Cumplimiento de instrucciones formales (estructura, citas, anexos).  


## Instrucciones Generales del Notebook  
1. Responder cada apartado en orden, combinando explicaciones en Markdown y código Python ejecutable.  
2. Mantener buenas prácticas: funciones reutilizables, pipelines y comentarios claros.  
3. Justificar todas las decisiones técnicas (por ejemplo, manejo de desbalance o selección de K en clustering).  
4. Incluir análisis ético y métricas de fairness en todos los apartados relevantes.  
5. El notebook debe funcionar como guion técnico y narrativo, sirviendo de base para el informe académico.  


## Estructura de Carpetas del Proyecto  
- `/data/` → datasets utilizados.  
- `/src/` → funciones reutilizables.  
- `/reports/figures/` → visualizaciones e imágenes.  
- `/reports/tables/` → tablas de resultados exportadas en `.csv`.  
- `/models/` → artefactos entrenados (opcional).  

Archivos adicionales:  
- `requirements.txt` → dependencias de Python para reproducibilidad.  
- `README.md` → instrucciones de ejecución.  


## Entregables Finales  
- Notebook ejecutable (`.ipynb`) con todos los apartados desarrollados.  
- Informe académico en formato APA (2.500–7.500 palabras, sin código).  
- Tablas y figuras exportadas desde el notebook.  


## Checklist de Cumplimiento  
- EDA completo: limpieza, calidad, distribuciones, correlaciones.  
- Dos o más modelos supervisados con validación cruzada y métricas (PR-AUC, ROC, Recall, Precision, F1).  
- Clustering justificado (Elbow, Silhouette y Davies-Bouldin) con descripción de perfiles.  
- Fairness: desempeño por subgrupos (sexo, educación, estado civil).  
- Conclusiones aplicables a un contexto crediticio real.  


### Instrucciones
- **EDA completo**: limpieza, calidad, outliers, subgrupos, correlaciones.  
- **≥2 modelos supervisados**: validación cruzada, **PR-AUC** (principal), ROC-AUC, Recall, Precision, F1.  
- **Clustering**: justificar **K** (Elbow + Silhouette + Davies-Bouldin), describir perfiles.  
- **Fairness**: desempeño por subgrupos (sexo, educación, estado civil).  
- **Conclusiones**: conectadas al problema real y a la aplicabilidad.

## Parte 02 — Configuración del entorno y **reproducibilidad**

**Qué haremos**  
- Importar librerías base y crear la estructura de carpetas.  
- Establecer **configuración de reproducibilidad** (semilla fija) para que particiones y resultados sean consistentes.

**Por qué**  
- Evita resultados distintos entre ejecuciones y facilita la revisión del docente.

**Qué esperamos obtener**  
- Mensajes de confirmación de versiones y rutas creadas (no hay métricas aún).

In [ ]:
import os, sys, random
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Configuración de reproducibilidad
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

# Estructura de carpetas
BASE_DIR = Path().resolve()
DATA_DIR = BASE_DIR / "data"
FIG_DIR = BASE_DIR / "reports" / "figures"
TAB_DIR = BASE_DIR / "reports" / "tables"
for d in (DATA_DIR, FIG_DIR, TAB_DIR):
    d.mkdir(parents=True, exist_ok=True)

print("Python:", sys.version.split()[0])
print("NumPy:", np.__version__, "| Pandas:", pd.__version__)
print("Rutas listas ->", "DATA:", DATA_DIR, "| FIG:", FIG_DIR, "| TAB:", TAB_DIR)

## Parte 03 — Carga de datos y **comprobaciones iniciales**

**Qué haremos**  
- Cargar el CSV original desde `./data/credit_default.csv`.  
- Verificar dimensiones, tipos y primeras filas.  
- Definir columna objetivo `default.payment.next.month`.

**Por qué**  
- Confirmar que el archivo es el correcto y que las columnas esperadas existen.

**Qué guardar para el informe**  
- Ninguna figura aún; solo constatación de que el dataset se cargó correctamente.

In [ ]:
csv_path = DATA_DIR / "credit_default.csv"
assert csv_path.exists(), f"Coloca el CSV en {csv_path} y vuelve a ejecutar."
df = pd.read_csv(csv_path)
target_col = "default.payment.next.month"

print("Shape:", df.shape)
display(df.head())
display(df.info())

## Parte 04 — **Caracterización del dataset** (para el informe)

**Fuente:** UCI Machine Learning Repository — *Default of Credit Card Clients (Taiwan, 2005)*.  
**Tamaño:** ≈30.000 registros, **23 variables** + **target**.  
**Estructura de variables:**  
- **Demográficas:** `SEX`, `EDUCATION`, `MARRIAGE`, `AGE`  
- **Financieras:** `LIMIT_BAL`  
- **Comportamiento (6 meses previos):** `PAY_0–PAY_6`, `BILL_AMT1–6`, `PAY_AMT1–6`  
- **Target:** `default.payment.next.month` (1=default, 0=no default)

> **Notas para el informe:** es un dataset **abierto**, sin PII, usado como benchmark académico; limita su generalización a un país/periodo.

*(Sin código en esta sección)*

## Parte 05 — Control de calidad y **limpieza mínima**

**Qué haremos**  
- Revisar duplicados y faltantes.  
- Corregir **categorías inválidas**: `EDUCATION` (0/5/6→4 “otros”) y `MARRIAGE` (0→3 “otros”).

**Por qué**  
- Dejar la data **consistente** antes de EDA/modelado.  

**Qué guardar**  
- Tabla de faltantes (si existieran) para el anexo.

In [ ]:
# Duplicados y faltantes
print("Duplicados:", df.duplicated().sum())
nulls = df.isna().sum().sort_values(ascending=False)
if (nulls>0).any():
    nulls.to_csv(TAB_DIR / "eda_missing_count.csv")
    display(nulls[nulls>0].head(10))

# Correcciones de categorías
if "EDUCATION" in df.columns:
    df["EDUCATION"] = df["EDUCATION"].replace({0:4, 5:4, 6:4})
if "MARRIAGE" in df.columns:
    df["MARRIAGE"] = df["MARRIAGE"].replace({0:3})

## Parte 06 — **EDA** (Exploratory Data Analysis)

**Qué haremos**  
1. Medir **tasa global de default** y **desbalance**.  
2. Estimar **tasas por subgrupos** (sexo, educación, matrimonio) y por **deciles de `LIMIT_BAL`**.  
3. Calcular **correlaciones Spearman** para variables numéricas.  
4. Guardar **tablas y figuras** para el informe.

**Por qué**  
- Entender el problema, detectar sesgos y elegir métricas adecuadas (**PR-AUC** para desbalance).

**Qué guardar**  
- `eda_target_rate.csv` · `eda_rate_by_[col].csv` · `eda_corr_spearman.csv` · `eda_corr_spearman.png`.

In [ ]:
# 1) Tasa global de default
rate = df[target_col].mean()
pd.DataFrame({"target_rate":[rate]}).to_csv(TAB_DIR / "eda_target_rate.csv", index=False)
print(f"Tasa global de default: {rate:.3f}")

# 2) Tasas por subgrupos
def rate_by(col):
    t = df.groupby(col)[target_col].mean().sort_values(ascending=False)
    t.to_csv(TAB_DIR / f"eda_rate_by_{col}.csv")
    return t

for col in ["SEX","EDUCATION","MARRIAGE"]:
    if col in df.columns:
        print(f"Tasa por {col}:"); display(rate_by(col))

# Deciles de LIMIT_BAL
if "LIMIT_BAL" in df.columns:
    df["_lim_decile"] = pd.qcut(df["LIMIT_BAL"], 10, labels=False, duplicates="drop")
    display(rate_by("_lim_decile"))
    df.drop(columns=["_lim_decile"], inplace=True)

# 3) Correlaciones Spearman
num_cols = [c for c in df.columns if pd.api.types.is_numeric_dtype(df[c])]
corr = df[num_cols].corr(method="spearman")
corr.to_csv(TAB_DIR / "eda_corr_spearman.csv")

# 4) Figura (sin seaborn)
plt.figure(figsize=(8,6))
plt.imshow(corr, aspect="auto")
plt.colorbar(); plt.title("Matriz de correlaciones (Spearman)")
plt.tight_layout(); plt.savefig(FIG_DIR / "eda_corr_spearman.png"); plt.close()

> **Interpretación esperada (completar tras ejecutar):**  
> - El atraso reciente (`PAY_0`) debiese correlacionar fuertemente con el target.  
> - `LIMIT_BAL` muestra gradiente de riesgo (mayor límite → menor default).  
> - Los subgrupos pueden exhibir diferencias → se justifican métricas de **fairness** más adelante.

## Parte 07 — **Ingeniería de características** (features derivadas)

**Qué haremos**  
- Crear variables **estables** y explicables:  
  - Tendencia de atraso: `pay_max`, `pay_mean`, `pay_count_late`.  
  - Esfuerzo de pago: `pay_effort_mean` = promedio `PAY_AMT`/`BILL_AMT`.  
  - Uso de línea: `utilization_mean` = promedio `BILL_AMT`/`LIMIT_BAL`.

**Por qué**  
- Capturan señales de comportamiento que **mejoran** el modelo y el clustering.

**Qué guardar**  
- `feat_corr_with_target.csv` (correlación de derivadas con el target).

In [ ]:
df_feat = df.copy()

pay_cols = [c for c in df.columns if c.startswith("PAY_")]
bill_cols = [c for c in df.columns if c.startswith("BILL_AMT")]
pamt_cols = [c for c in df.columns if c.startswith("PAY_AMT")]

# Tendencias de atraso
if pay_cols:
    df_feat["pay_max"] = df[pay_cols].max(axis=1)
    df_feat["pay_mean"] = df[pay_cols].mean(axis=1)
    df_feat["pay_count_late"] = (df[pay_cols] > 0).sum(axis=1)

# Esfuerzo de pago y uso de línea
import numpy as np
if bill_cols and pamt_cols:
    ratios = []
    for bcol, pcol in zip(sorted(bill_cols), sorted(pamt_cols)):
        denom = df[bcol].replace(0, np.nan).abs()
        ratios.append(df[pcol] / denom)
    df_feat["pay_effort_mean"] = pd.concat(ratios, axis=1).mean(axis=1, skipna=True)

if "LIMIT_BAL" in df.columns and bill_cols:
    df_feat["utilization_mean"] = df[bill_cols].divide(df["LIMIT_BAL"].replace(0, np.nan), axis=0).mean(axis=1, skipna=True)

# Correlaciones de derivadas con el target
derived = [c for c in ["pay_max","pay_mean","pay_count_late","pay_effort_mean","utilization_mean"] if c in df_feat.columns]
if derived:
    corr_deriv = df_feat[derived + [target_col]].corr(method="spearman")[target_col].sort_values(ascending=False)
    corr_deriv.to_csv(TAB_DIR / "feat_corr_with_target.csv")
    display(corr_deriv)

> **Buenas prácticas**  
> - Evitar derivadas que usen información del **futuro** (fuga).  
> - Documentar cálculo y **sentido de negocio** de cada feature.

## Parte 08 — Partición y **pipelines** (sin fuga de información)

**Qué haremos**  
- Split **train/test estratificado**.  
- `ColumnTransformer` con imputación y escalado (numéricas) + one-hot (categóricas).  
- Encapsular preprocesamiento en **Pipeline**.

**Por qué**  
- Los pasos de preparación deben aplicarse **solo** con datos de **train** durante la validación (evita fuga).

**Qué guardar**  
- No se exporta nada aquí; sí se imprimen formas y tasas del target en train/test.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

y = df_feat[target_col].astype(int)
X = df_feat.drop(columns=[target_col])

num_cols = [c for c in X.columns if pd.api.types.is_numeric_dtype(X[c])]
cat_cols = [c for c in X.columns if c not in num_cols]

numeric_pipe = Pipeline([("imp", SimpleImputer(strategy="median")), ("sc", StandardScaler())])
categorical_pipe = Pipeline([("imp", SimpleImputer(strategy="most_frequent")), ("ohe", OneHotEncoder(handle_unknown="ignore", sparse_output=False))])

preprocess = ColumnTransformer([("num", numeric_pipe, num_cols), ("cat", categorical_pipe, cat_cols)], remainder="drop")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=RANDOM_SEED)

print("Shapes train/test:", X_train.shape, X_test.shape)
print("Target rate train/test:", round(y_train.mean(),3), round(y_test.mean(),3))

## Parte 09 — **Modelado supervisado** (clasificación)

**Qué haremos**  
- Entrenar **Regresión Logística** y **Árbol de Decisión** con **CV k=5** (estratificada).  
- Métrica principal: **PR-AUC** (adecuada con desbalance).  
- Reportar también ROC-AUC, F1, Recall, Precision.  
- Guardar métricas y curvas PR/ROC del **mejor modelo**.

**Por qué**  
- Comparar modelos de distinta naturaleza (lineal vs no lineal) y seleccionar el que mejor **recall en alta precisión** logre.

**Qué guardar**  
- `supervisado_metricas_test.csv` · figuras `roc_*.png` y `pr_*.png` · `cm_*_thr05.png`.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import average_precision_score, roc_auc_score, f1_score, precision_score, recall_score, confusion_matrix, RocCurveDisplay, PrecisionRecallDisplay

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)

# 1) Logistic Regression
log_pipe = Pipeline([("prep", preprocess), ("clf", LogisticRegression(max_iter=1000, class_weight="balanced"))])
log_grid = GridSearchCV(log_pipe, {"clf__C":[0.1,1,10]}, scoring="average_precision", cv=cv, n_jobs=-1)
log_grid.fit(X_train, y_train)

# 2) Decision Tree
tree_pipe = Pipeline([("prep", preprocess), ("clf", DecisionTreeClassifier(class_weight="balanced", random_state=RANDOM_SEED))])
tree_grid = GridSearchCV(tree_pipe, {"clf__max_depth":[3,5,7,9], "clf__min_samples_leaf":[1,5,20]}, scoring="average_precision", cv=cv, n_jobs=-1)
tree_grid.fit(X_train, y_train)

def eval_on_test(estimator, name):
    proba = estimator.predict_proba(X_test)[:,1]
    preds = (proba>=0.5).astype(int)
    return {
        "model": name,
        "PR-AUC": average_precision_score(y_test, proba),
        "ROC-AUC": roc_auc_score(y_test, proba),
        "F1": f1_score(y_test, preds),
        "Recall": recall_score(y_test, preds),
        "Precision": precision_score(y_test, preds)
    }, proba

log_metrics, log_proba = eval_on_test(log_grid.best_estimator_, "LogisticRegression")
tree_metrics, tree_proba = eval_on_test(tree_grid.best_estimator_, "DecisionTree")

import pandas as pd
metrics_df = pd.DataFrame([log_metrics, tree_metrics]).sort_values("PR-AUC", ascending=False)
metrics_df.to_csv(TAB_DIR / "supervisado_metricas_test.csv", index=False)
display(metrics_df)

best_name = metrics_df.iloc[0]["model"]
best_estimator = log_grid.best_estimator_ if best_name=="LogisticRegression" else tree_grid.best_estimator_

RocCurveDisplay.from_estimator(best_estimator, X_test, y_test); plt.title(f"ROC — {best_name}")
plt.savefig(FIG_DIR / f"roc_{best_name}.png"); plt.close()

PrecisionRecallDisplay.from_estimator(best_estimator, X_test, y_test); plt.title(f"PR — {best_name}")
plt.savefig(FIG_DIR / f"pr_{best_name}.png"); plt.close()

cm = confusion_matrix(y_test, (best_estimator.predict_proba(X_test)[:,1] >= 0.5).astype(int))
plt.figure(); plt.imshow(cm); plt.title(f"Matriz de confusión — {best_name} (umbral=0.5)"); plt.colorbar(); plt.tight_layout()
plt.savefig(FIG_DIR / f"cm_{best_name}_thr05.png"); plt.close()

# Guardamos para fairness/umbral
_best_proba = log_proba if best_name=="LogisticRegression" else tree_proba

## Parte 10 — **Optimización de umbral** (opcional recomendado)

**Qué haremos**  
- Buscar el **umbral de decisión** que maximiza **F1** (o la métrica/costo que definamos).  
- Guardar la nueva matriz de confusión y métricas al umbral óptimo.

**Por qué**  
- En contextos desbalanceados, mover el umbral **mejora** el equilibrio Recall/Precision.

**Qué guardar**  
- `supervisado_metricas_umbral_opt.csv` · `cm_*_thr_opt.png`.

In [ ]:
import numpy as np
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix

def best_threshold(y_true, y_proba, metric="f1"):
    best_thr, best_val = 0.5, -1.0
    for thr in np.linspace(0.05, 0.95, 19):
        preds = (y_proba >= thr).astype(int)
        val = f1_score(y_true, preds) if metric=="f1" else f1_score(y_true, preds)
        if val > best_val:
            best_thr, best_val = thr, val
    return best_thr, best_val

thr, val = best_threshold(y_test, _best_proba, metric="f1")
preds_opt = (_best_proba >= thr).astype(int)

cm = confusion_matrix(y_test, preds_opt)
plt.figure(); plt.imshow(cm); plt.title(f"Matriz de confusión (umbral={thr:.2f})"); plt.colorbar(); plt.tight_layout()
plt.savefig(FIG_DIR / "cm_best_thr_opt.png"); plt.close()

pd.DataFrame([{
    "threshold": thr,
    "F1": f1_score(y_test, preds_opt),
    "Recall": recall_score(y_test, preds_opt),
    "Precision": precision_score(y_test, preds_opt)
}]).to_csv(TAB_DIR / "supervisado_metricas_umbral_opt.csv", index=False)

## Parte 11 — **Fairness por subgrupos**

**Qué haremos**  
- Evaluar **Recall, Precision, FPR, FNR** por `SEX`, `EDUCATION`, `MARRIAGE` usando el **umbral óptimo**.

**Por qué**  
- Identificar disparidades en desempeño y documentarlas éticamente.

**Qué guardar**  
- `fairness_SEX.csv` · `fairness_EDUCATION.csv` · `fairness_MARRIAGE.csv`.

In [ ]:
def subgroup_table(df_base, y_true, y_proba, subgroup_col, thr):
    preds = (y_proba >= thr).astype(int)
    rows = []
    for g, idx in df_base.groupby(subgroup_col).groups.items():
        yt = y_true.loc[idx]
        pr = preds[idx]
        fp = ((pr==1) & (yt==0)).sum()
        fn = ((pr==0) & (yt==1)).sum()
        tn = ((pr==0) & (yt==0)).sum()
        tp = ((pr==1) & (yt==1)).sum()
        rec = tp/(tp+fn) if (tp+fn)>0 else float("nan")
        pre = tp/(tp+fp) if (tp+fp)>0 else float("nan")
        fpr = fp/(fp+tn) if (fp+tn)>0 else float("nan")
        fnr = fn/(fn+tp) if (fn+tp)>0 else float("nan")
        rows.append({"grupo": g, "n": len(idx), "Recall": rec, "Precision": pre, "FPR": fpr, "FNR": fnr})
    return pd.DataFrame(rows).sort_values("grupo")

thr = float(pd.read_csv(TAB_DIR / "supervisado_metricas_umbral_opt.csv")["threshold"].iloc[0])

for col in ["SEX","EDUCATION","MARRIAGE"]:
    if col in df_feat.columns:
        t = subgroup_table(df_feat, y_test, _best_proba, col, thr)
        t.to_csv(TAB_DIR / f"fairness_{col}.csv", index=False)
        print(f"Fairness por {col}:"); display(t)

## Parte 12 — **Clustering** (no supervisado)

**Qué haremos**  
- KMeans con K=2..8 usando variables derivadas **estables**.  
- Selección de K con **Elbow + Silhouette + Davies-Bouldin** y visualización **PCA 2D**.  
- **Post-hoc**: relacionar clúster con tasa de default (el target no forma los clústeres).

**Por qué**  
- Caracterizar perfiles de riesgo para estrategias diferenciadas (no reemplaza al modelo supervisado).

**Qué guardar**  
- `clustering_k_metrics.csv` · `clusters_perfil.csv` · `clusters_default_rate.csv` · `clusters_pca2.png`.

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score
from sklearn.decomposition import PCA

cluster_vars = [c for c in ["pay_mean","pay_max","pay_count_late","pay_effort_mean","utilization_mean"] if c in df_feat.columns]
assert cluster_vars, "No hay variables derivadas para clustering (revisa Parte 07)."

Z = df_feat[cluster_vars].copy().astype(float).fillna(df_feat[cluster_vars].median())
Z = StandardScaler().fit_transform(Z)

rows = []
for K in range(2, 9):
    km = KMeans(n_clusters=K, random_state=RANDOM_SEED, n_init=10)
    labels = km.fit_predict(Z)
    sil = silhouette_score(Z, labels)
    db  = davies_bouldin_score(Z, labels)
    rows.append({"K":K, "silhouette":sil, "davies_bouldin":db, "inertia":km.inertia_})
k_tbl = pd.DataFrame(rows)
k_tbl.to_csv(TAB_DIR / "clustering_k_metrics.csv", index=False)
display(k_tbl)

# Selección sugerida (interpretación manda)
K_best = int(k_tbl.sort_values(["silhouette","davies_bouldin"], ascending=[False, True]).iloc[0]["K"])
km = KMeans(n_clusters=K_best, random_state=RANDOM_SEED, n_init=10)
labels = km.fit_predict(Z)
df_feat["cluster"] = labels

perfil = df_feat.groupby("cluster")[cluster_vars].agg(["mean","median"]).round(3)
perfil.to_csv(TAB_DIR / "clusters_perfil.csv")
display(perfil)

# PCA 2D
pca2 = PCA(n_components=2, random_state=RANDOM_SEED).fit_transform(Z)
pca_df = pd.DataFrame(pca2, columns=["PC1","PC2"]); pca_df["cluster"] = labels
pca_df.to_csv(TAB_DIR / "clusters_pca2.csv", index=False)

plt.figure()
for k in sorted(pca_df["cluster"].unique()):
    pts = pca_df[pca_df["cluster"]==k]
    plt.scatter(pts["PC1"], pts["PC2"], s=10, label=f"C{k}")
plt.legend(); plt.title("PCA 2D — clusters"); plt.tight_layout()
plt.savefig(FIG_DIR / "clusters_pca2.png"); plt.close()

# Post-hoc: tasa de default por cluster
t_default = df_feat.groupby("cluster")[target_col].mean().rename("default_rate").to_frame()
t_default.to_csv(TAB_DIR / "clusters_default_rate.csv")
display(t_default)

## Parte 13 — **Integración de resultados** y conclusiones

**Qué haremos**  
- Comparar **modelo ganador vs baseline** (mejora en PR-AUC y Recall).  
- Resumir **segmentos/clústeres de riesgo** con tasa de default.  
- Redactar **conclusiones y limitaciones** para el informe.

**Qué guardar**  
- `integracion_supervisado.csv` + tablas de clústeres ya generadas.

In [ ]:
import pandas as pd
sup_metrics = pd.read_csv(TAB_DIR / "supervisado_metricas_test.csv")
best_row = sup_metrics.sort_values("PR-AUC", ascending=False).iloc[0]
baseline_row = sup_metrics.sort_values("PR-AUC", ascending=False).iloc[-1]

def pct_gain(a, b):
    return (a - b) / b * 100 if b != 0 else float("nan")

comparativa = pd.DataFrame([{
    "metric": "PR-AUC",
    "winner": best_row["PR-AUC"],
    "baseline": baseline_row["PR-AUC"],
    "gain_%": pct_gain(best_row["PR-AUC"], baseline_row["PR-AUC"])
},{
    "metric": "Recall",
    "winner": best_row["Recall"],
    "baseline": baseline_row["Recall"],
    "gain_%": pct_gain(best_row["Recall"], baseline_row["Recall"])
}])
comparativa.to_csv(TAB_DIR / "integracion_supervisado.csv", index=False)
display(comparativa)

# Mostrar tablas de clústeres si existen
for p in [TAB_DIR / "clusters_perfil.csv", TAB_DIR / "clusters_default_rate.csv"]:
    if p.exists():
        print("Tabla disponible:", p.name); display(pd.read_csv(p))

### **Conclusiones (completar tras ejecutar)**
- **Modelo ganador** y variables explicativas principales (mencionar señales: atraso reciente, uso de línea, etc.).  
- **Segmentos de riesgo** a priorizar (alto atraso/baja capacidad vs bajo riesgo).  
- **Limitaciones**: país/periodo único; variables acotadas; posible sesgo.  
- **Trabajo futuro**: costos FP/FN, validación externa, features temporales avanzadas.

> **Checklist antes de entregar**  
> - [ ] Todas las tablas/figuras están en `/reports/`.  
> - [ ] Se cita explícitamente la **fuente UCI**.  
> - [ ] El informe APA usa estas tablas/figuras (sin pegar código).